In [1]:
import sqlite3
import json

In [2]:
conn = sqlite3.connect('../databases/CovidData.db')
cursor = conn.cursor()

In [3]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS covid_journals_data (
        id TEXT PRIMARY KEY,
        source TEXT,
        pub_date TEXT,
        pub_timestamp TEXT,
        title TEXT,
        link TEXT,
        authors TEXT,
        authors_affiliations TEXT,
        keywords TEXT
    )
''')

In [ ]:
from datetime import datetime
with open('../data/combined_sources.json', 'r') as json_file:
    data = json.load(json_file)
    list_delimiter = ", "

    for item in data:
        if item['source'] == 'Doaj':
            # format journal date
            datetime_object = datetime.strptime(
                item['date'], '%Y-%m-%dT%H:%M:%SZ')
            datetime_timestamp = datetime.timestamp(datetime_object)

            # format authors and affiliations lists
            authors_list = []
            author_affiliations_list = []
            for author in item['authors']:
                authors_list.append(author.get('name', ""))
                author_affiliations_list.append(author.get('affiliation', ""))

            # insert db row
            cursor.execute("INSERT OR IGNORE INTO covid_journals_data VALUES(?,?,?,?,?,?,?,?,?)", (
                item['id'],
                item['source'],
                str(datetime_object),
                str(datetime_timestamp),
                item['title'],
                list_delimiter.join([link['url'] for link in item['link']]),
                list_delimiter.join(authors_list),
                list_delimiter.join(author_affiliations_list),
                list_delimiter.join(item['keywords'])
            ))
        if item['source'] == 'PubMed':
            # format journal date
            formatted_date_str = f"{item['date']['Year']}-{item['date']['Month']}-{item['date']['Day']}"
            datetime_object = datetime.strptime(formatted_date_str, '%Y-%m-%d')
            datetime_timestamp = datetime.timestamp(datetime_object)

            # format authors and affiliations lists
            authors_list = []
            affiliations = []
            if isinstance(item['authors'], list):
                for author in item['authors']:
                    if author.get('AffiliationInfo'):
                        if isinstance(author['AffiliationInfo'], list):
                            for affiliation in author.get('AffiliationInfo', []):
                                affiliations.append(affiliation.get('Affiliation', ''))
                        elif isinstance(author['AffiliationInfo'], dict):
                            affiliations.append(author.get(
                                'AffiliationInfo', {}).get('Affiliation', ''))
                    authors_list.append(
                        f"{author.get('ForeName', '')} {author.get('LastName', '')}")

            # format keywords lists
            keywords_list = []
            for keyword in item.get('keywords', []):
                keywords_list.append(keyword['#text'])

            # insert db row
            cursor.execute("INSERT OR IGNORE INTO covid_journals_data VALUES(?,?,?,?,?,?,?,?,?)", (
                item['id'],
                item['source'],
                str(datetime_object),
                str(datetime_timestamp),
                item['title'],
                item['link'],
                list_delimiter.join(authors_list),
                list_delimiter.join(affiliations),
                list_delimiter.join(keywords_list)
            ))

{'Affiliation': 'Division for Health Services, Norwegian Institute of Public Health, Oslo, Norway. andrew.garratt@fhi.no.'}
{'Affiliation': 'Health Services Research Unit, Akershus University Hospital, Nordbyhagen, Norway. andrew.garratt@fhi.no.'}
{'Affiliation': 'Health Services Research Unit, Akershus University Hospital, Nordbyhagen, Norway.'}
{'Affiliation': 'Institute of Clinical Medicine, University of Oslo, Oslo, Norway.'}
{'Affiliation': 'Department of Pulmonary Medicine, Medical Division, Akershus University Hospital, Nordbyhagen, Norway.'}


TypeError: sequence item 0: expected str instance, dict found

In [9]:
conn.commit()
conn.close()